In [ ]:
from langchain.llms import GooglePalm
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Annoy
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# get this free api key from https://makersuite.google.com/

api_key = 'your_api_key_here' 

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [ ]:
# Read CSV file 
loader = CSVLoader(file_path='health_lifestyle_nutrition_dataset.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

In [ ]:
embedding_function = HuggingFaceInstructEmbeddings()

text = "This is a test line."
query_result = embedding_function.embed_query(text)

In [ ]:
# Store data into vector database by providing embedding technique
vector_store_from_docs = Annoy.from_documents(data, embedding_function)

In [ ]:
vector_store_from_docs

In [ ]:
# Create a retriver object which return document with minimal 0.7 cosine similarity
retriever = vector_store_from_docs.as_retriever(score_threshold = 0.7)

In [ ]:
rdocs = retriever.get_relevant_documents("What are the benefits of  regular exercise?")
rdocs

In [ ]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

# create RetrievalQA chain object, which return answer based on hyperparameter.
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain('What are the benefits of  regular exercise?')